In [18]:
import torch
import torch.nn as nn
from torchtext import datasets
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim

In [2]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url)
passengers = data.Passengers.values.astype(np.float32)
data.Month[0]


'1949-01'

In [3]:
class AirlineDataset(Dataset):
    def __init__(self, transform=None):
        super().__init__()
        URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
        self.data = pd.read_csv(URL)
        self.passengers = self.data.Passengers
        self.transform = transform

    
    def __getitem__(self, index):
        month, passenger = self.data.Month[index], self.data.Passengers[index]
        return month, passenger
    
    def __len__(self):
        return len(self.data)


In [4]:
dataset = AirlineDataset()
loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [39]:
class DinoNameDataset(Dataset):
    def __init__(self, transform=None, sos_key='<SOS>', eos_key='<EOS>', pad_key='<PAD>'):
        super().__init__()
        self.transform = transform
        # file_path = "/Users/musazenbilci/Desktop/mosesopposite/DeepLearningAI-RNN/W1A2/dinos.txt"
        file_path = "/Users/musazenbilci/Desktop/mosesopposite/andrej/makemore/tr_names.txt"
        with open(file_path, 'r') as file:
            self.data = file.readlines()
        self.max_len = len(max(self.data, key=len))
        self.sos_key = sos_key
        self.eos_key = eos_key
        self.pad_key = pad_key
        self.alphabet = [self.sos_key] + list(set(''.join(self.data).lower())) + [self.eos_key, self.pad_key] 
        self.i2l = {idx: letter for idx, letter in enumerate(self.alphabet) }
        self.l2i = {letter: torch.tensor(idx) for idx, letter in enumerate(self.alphabet) }
        self.VOCAB_SIZE = len(self.l2i)
    
    def __getitem__(self, index):
        data = self.tokenize(self.data[index].lower()) # if self.transform  else self.data[index]
        return data
    
    def __len__(self):
        return len(self.data)
    
    def tokenize(self, x):
        return torch.tensor([self.l2i[self.sos_key]] + [self.l2i[c.lower()] if c!='\n' else self.l2i[self.eos_key] for c in list(x)])


In [40]:
ddataset = DinoNameDataset()
alphabet = ddataset.alphabet
i2l = ddataset.i2l
l2i = ddataset.l2i
VOCAB_SIZE = ddataset.VOCAB_SIZE

In [41]:
'i̇' in ddataset.alphabet

False

In [42]:
# alphabet = ['<SOS>'] + ddataset.alphabet + ['<EOS>', '<PAD>'] 
# i2l = {idx: letter for idx, letter in enumerate(alphabet) }
# l2i = {letter: torch.tensor(idx) for idx, letter in enumerate(alphabet) }
# VOCAB_SIZE = len(l2i)

In [43]:
def pad_collate(batch):
    sequences = batch[:]
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=l2i['<PAD>'])
    return padded_sequences

In [44]:
train_data, val_data, test_data = random_split(ddataset, [0.8, 0.1, 0.1])

In [45]:
dloader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=pad_collate)
val_loader = DataLoader(val_data, batch_size=4, shuffle=False, collate_fn=pad_collate)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=pad_collate)

In [46]:
# tokenize = lambda a: torch.tensor([l2i['<SOS>']] + [l2i[c.lower()] if c!='\n' else l2i['<EOS>'] for c in list(a)])

In [47]:
len(dloader)
for word in dloader:
    print(word.shape)
#     # print(word)
    break

torch.Size([4, 14])


<img src="../images/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">

In [ ]:
class RNNCell(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.linear1 = nn.Linear(vocab_size, vocab_size, bias=False) # Waa
        self.linear2 = nn.Linear(vocab_size, vocab_size, bias=True) # Wax + ba
        self.tanh = nn.Tanh()

        self.final = nn.Linear(vocab_size, vocab_size, bias=True) # Wya + by

    def forward(self, x, previous_a): # also called hidden state
        aa = self.linear1(previous_a)
        ax = self.linear2(x)
        x = torch.add(aa, ax)
        a = self.tanh(x)
        x = self.final(a)
        return x, a


In [229]:
cell = RNNCell(VOCAB_SIZE)

<img src="/Users/musazenbilci/Desktop/mosesopposite/DeepLearningAI-RNN/W1A1/images/RNN.png" style="width:500;height:300px;">

In [49]:
make_one_hot = lambda x: torch.nn.functional.one_hot(x, VOCAB_SIZE).to(torch.float32)
make_zero_hot = lambda : torch.zeros(VOCAB_SIZE).to(torch.float32)

In [85]:
from torch import float32

initial_lr = 1e-2
cell = RNNCell(VOCAB_SIZE)
optimizer = optim.SGD(cell.parameters(), lr=initial_lr)
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=8, threshold=5e-2)
a0 = make_zero_hot()

EPOCH=100

for ep in range(EPOCH):
    epoch_loss = 0
    for batch in dloader:
        batch_loss = 0
        for word in batch:
            a_prev = a0
            loss = 0
            optimizer.zero_grad()
            for ci in range(len(word)-1):
                if word[ci] == l2i[ddataset.sos_key]:
                    continue
                
                onehot = make_one_hot(word[ci])
                logits, act = cell(onehot, a_prev)
                loss += criterion(logits, word[ci+1])
                a_prev = act

                if word[ci] == l2i[ddataset.eos_key]: # skip paddings
                    break
            # print(loss.item())
            loss.backward()
            optimizer.step()
            batch_loss += loss.item()
            # break
        # print("BATCH_LOSS", batch_loss / 4)
        epoch_loss += batch_loss / 4
        # break
    print("EPOCH_LOSS", epoch_loss / 384)
    scheduler.step(epoch_loss / 384, ep)
    if initial_lr != optimizer.param_groups[0]['lr']:
        print(f"LR Changed {optimizer.param_groups[0]['lr']}")
        initial_lr = optimizer.param_groups[0]['lr']
    

EPOCH_LOSS 20.82814035192132
EPOCH_LOSS 16.60375009709969
EPOCH_LOSS 15.394904699642211
EPOCH_LOSS 14.71077059286957
EPOCH_LOSS 14.165701747406274
EPOCH_LOSS 13.823744678637013
EPOCH_LOSS 13.589067362093678
EPOCH_LOSS 13.385934456018731
EPOCH_LOSS 13.172507057975357
EPOCH_LOSS 13.112891293053204
EPOCH_LOSS 13.012882166852554
EPOCH_LOSS 13.000476781278849
EPOCH_LOSS 12.83463465794921
EPOCH_LOSS 12.730492672494924
EPOCH_LOSS 12.686102047174549
EPOCH_LOSS 12.543046530957023
EPOCH_LOSS 12.524487066470707
EPOCH_LOSS 12.681524048559368
EPOCH_LOSS 12.471853277646005
EPOCH_LOSS 12.392977533396333
EPOCH_LOSS 12.39987679120774
EPOCH_LOSS 12.896046592155471
EPOCH_LOSS 12.794668367520595
EPOCH_LOSS 12.614421548089013
LR Changed 0.001
EPOCH_LOSS 11.416940778804323
EPOCH_LOSS 10.976396360434592
EPOCH_LOSS 10.818512260603407
EPOCH_LOSS 10.714353054917106
EPOCH_LOSS 10.614205507561564
EPOCH_LOSS 10.543386969560137
EPOCH_LOSS 10.481041078145305
EPOCH_LOSS 10.454274164667973
EPOCH_LOSS 10.38978227352102

In [54]:
torch.save(cell.state_dict, './turkish_rnn_cell_loss9.pth')

In [ ]:
# torch.save(cell.state_dict, './rnn_cell_loss16.pth')

In [115]:
a00 = a0
a00[0] = 0
a00 = a00.to(float32)
print(a0.dtype)

torch.int64


In [82]:
output = ''
# input, a_prev = a0.to(float32), a00.to(float32)
test_ind = l2i['z']
test_ind2 = l2i['b']
input, a_prev = make_one_hot(torch.tensor(test_ind)), make_zero_hot() # make_one_hot(torch.tensor(test_ind2))
# output += i2l[test_ind2.item()]
output += i2l[test_ind.item()]
while True:
    logits, act = cell(input, a_prev)
    probs = torch.softmax(logits, 0)
    chosen = torch.argmax(probs)
    if chosen == l2i['<EOS>']:
        break
    output += i2l[chosen.item()]
    input = torch.nn.functional.one_hot(chosen, VOCAB_SIZE).to(float32)
    a_prev = act
output

/var/folders/f9/kqml6rjn5g5688g1y7fn01ch0000gn/T/ipykernel_21005/894917494.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input, a_prev = make_one_hot(torch.tensor(test_ind)), make_zero_hot() # make_one_hot(torch.tensor(test_ind2))


'zahan'